In [8]:
# Fetch the data from Firebase

from firebase import firebase
import math
firebase = firebase.FirebaseApplication('https://androidbletutorial.firebaseio.com/',None)
All_Sample = []
result = firebase.get('',None)
length = 10

# Define Partition Function
def Partition(value):
    end_value = pow(2,16)-1  # maximam value
    global length
    num_level = length
    return_value = math.ceil(value*num_level/end_value)
    #print(value,"  ", return_value);
    return return_value

# Sample value
for key in result.keys():
  temp = result[key]["text"].split(":")[2]
  x1, x2 = temp.split(" ")[1], temp.split(" ")[2]
  y1, y2 = temp.split(" ")[3], temp.split(" ")[4]
  z1, z2 = temp.split(" ")[5], temp.split(" ")[6]

  x = int(x2) + (int(x1)<<8)
  y = int(y2) + (int(y1)<<8)
  z = int(z2) + (int(z1)<<8)

  sample = math.sqrt(x*x+y*y+z*z)
  sample = Partition(int(sample))
  All_Sample.append(sample)
Sample_list = ''.join([str(elem) for elem in All_Sample]) 
print(Sample_list)

# find transition matrix and emission matrix
#defining states and sequence symbols
import numpy as np
import decimal
from decimal import Decimal
import math
import time

#transition matrix
transition = np.array([[0.9998611,0.0001389],[0.0000347,0.9999653]])
#emission matrix
emission = np.array([[0.02,0.16,0.06,0.12,0.1,0.1,0.08,0.14,0.04,0.18],
                     [0.14,0.04,0.18,0.02,0.16,0.06,0.12,0.1,0.1,0.08]])


states = ['H','C']
states_dic = {'H':0, 'C':1}
#sequence_syms = {'1':0,'6600':1,'14000':2,'28000':3,'35000':4,'41000':5,'47000':6,'55000':7,'57000':8,'65000':9}
sequence_syms = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9}
sequence = ['0','1','2','3','4','5','6','7','8','9']
#sequence = ['1', '6600', '14000', '28000', '35000', '41000', '47000', '55000', '57000', '65000']
#test sequence
test_sequence = Sample_list

#probabilities of going to end state
end_probs = [0.1, 0.1]
#probabilities of going from start state
start_probs = [0.5, 0.5]


#function to find forward probabilities
def forward_probs():
    # node values stored during forward algorithm
    node_values_fwd = np.zeros((len(states), len(test_sequence)))

    for i, sequence_val in enumerate(test_sequence):
        for j in range(len(states)):
            # if first sequence value then do this
            if (i == 0):
                node_values_fwd[j, i] = start_probs[j] * emission[j, sequence_syms[sequence_val]]
            # else perform this
            else:
                values = [node_values_fwd[k, i - 1] * emission[j, sequence_syms[sequence_val]] * transition[k, j] for k in
                          range(len(states))]
                node_values_fwd[j, i] = sum(values)

    #end state value
    end_state = np.multiply(node_values_fwd[:, -1], end_probs)
    end_state_val = sum(end_state)
    return node_values_fwd, end_state_val


#function to find backward probabilities
def backward_probs():
    # node values stored during forward algorithm
    node_values_bwd = np.zeros((len(states), len(test_sequence)))

    #for i, sequence_val in enumerate(test_sequence):
    for i in range(1,len(test_sequence)+1):
        for j in range(len(states)):
            # if first sequence value then do this
            if (-i == -1):
                node_values_bwd[j, -i] = end_probs[j]
            # else perform this
            else:
                values = [node_values_bwd[k, -i+1] * emission[k, sequence_syms[test_sequence[-i+1]]] * transition[j, k] for k in range(len(states))]
                node_values_bwd[j, -i] = sum(values)

    #start state value
    start_state = [node_values_bwd[m,0] * emission[m, sequence_syms[test_sequence[0]]] for m in range(len(states))]
    start_state = np.multiply(start_state, start_probs)
    start_state_val = sum(start_state)
    return node_values_bwd, start_state_val


#function to find si probabilities
def si_probs(forward, backward, forward_val):

    si_probabilities = np.zeros((len(states), len(test_sequence)-1, len(states)))

    for i in range(len(test_sequence)-1):
        for j in range(len(states)):
            for k in range(len(states)):
                si_probabilities[j,i,k] = ( forward[j,i] * backward[k,i+1] * transition[j,k] * emission[k,sequence_syms[test_sequence[i+1]]] ) \
                                                    / forward_val
    return si_probabilities

#function to find gamma probabilities
def gamma_probs(forward, backward, forward_val):

    gamma_probabilities = np.zeros((len(states), len(test_sequence)))

    for i in range(len(test_sequence)):
        for j in range(len(states)):
            #gamma_probabilities[j,i] = ( forward[j,i] * backward[j,i] * emission[j,sequence_syms[test_sequence[i]]] ) / forward_val
            gamma_probabilities[j, i] = (forward[j, i] * backward[j, i]) / forward_val

    return gamma_probabilities

#performing iterations until convergence

for iteration in range(200):

    print('\nIteration No: ', iteration + 1)
    # print('\nTransition:\n ', transition)
    # print('\nEmission: \n', emission)

    #Calling probability functions to calculate all probabilities
    fwd_probs, fwd_val = forward_probs()
    bwd_probs, bwd_val = backward_probs()
    si_probabilities = si_probs(fwd_probs, bwd_probs, fwd_val)
    gamma_probabilities = gamma_probs(fwd_probs, bwd_probs, fwd_val)

    # print('Forward Probs:')
    # print(np.matrix(fwd_probs))
    
    # print('Backward Probs:')
    # print(np.matrix(bwd_probs))
    #
    # print('Si Probs:')
    # print(si_probabilities)

    # print('Gamma Probs:')
    # print(np.matrix(gamma_probabilities))

    #caclculating 'a' and 'b' matrices
    a = np.zeros((len(states), len(states)))
    b = np.zeros((len(states), len(sequence_syms)))

    #'a' matrix
    for j in range(len(states)):
        for i in range(len(states)):
            for t in range(len(test_sequence)-1):
                a[j,i] = a[j,i] + si_probabilities[j,t,i]

            denomenator_a = [si_probabilities[j, t_x, i_x] for t_x in range(len(test_sequence) - 1) for i_x in range(len(states))]
            denomenator_a = sum(denomenator_a)

            if (denomenator_a == 0):
                a[j,i] = 0
            else:
                a[j,i] = a[j,i]/denomenator_a

    #'b' matrix
    for j in range(len(states)): #states
        for i in range(len(sequence)): #seq
            indices = [idx for idx, val in enumerate(test_sequence) if val == sequence[i]]
            numerator_b = sum( gamma_probabilities[j,indices] )
            denomenator_b = sum( gamma_probabilities[j,:] )

            if (denomenator_b == 0):
                b[j,i] = 0
            else:
                b[j, i] = numerator_b / denomenator_b


    print('\nMatrix a:\n')
    print(np.matrix(a.round(decimals=4)))
    print('\nMatrix b:\n')
    print(np.matrix(b.round(decimals=4)))

    transition = a
    emission = b

    new_fwd_temp, new_fwd_temp_val = forward_probs()
    print('New forward probability: ', new_fwd_temp_val)
    diff =  np.abs(fwd_val - new_fwd_temp_val)
    print('Difference in forward probability: ', diff)

    if (diff < 0.00000001):
        break

9189210921061068681336229103863696597110210992102101051010225263332866188689310178199769621181717926269297999693893969373109889991936663981099638193977815259102266183391339111696396983636696103619721097102629563918758363959102106691918898668313316666351051989111159136683393103265586919383731319810979101095689993969910916915910106276331713963783883910316671177861983189191088791109863373323109261069971071062399199199199831061391996106399999683371939997910310231961032829783161068866109819386691031396109277393611091067391021062363625715102813933938197233836131769663381199681081013216326777109105686632676110375238995929593101028102110910691101061318106969310392365526279961098319789965910368889179166281055210272210910383103961065937333716102371571379939916951068771099333653106101095863910639383366108101071869210961067882105851010910816353871099922317616866929259363559626139228991999369391210928696187711019107319110231638839381119879911098983319751019396899581069939737102581036668711186936963

/home/shiv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in double_scalars
/home/shiv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in double_scalars



Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  4

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  5

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  6

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  7

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan 


Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  44

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  45

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  46

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  47

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan 


Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  82

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  83

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  84

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  85

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan 

New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  119

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  120

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  121

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  122

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  123

Matrix a:

[[


Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  158

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  159

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  160

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  161

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan 


Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  196

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  197

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  198

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
New forward probability:  nan
Difference in forward probability:  nan

Iteration No:  199

Matrix a:

[[nan nan]
 [nan nan]]

Matrix b:

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan 

In [44]:
from firebase import firebase
import math
firebase = firebase.FirebaseApplication('https://androidbletutorial.firebaseio.com/',None)

result=firebase.get('',None)
length = 10

# Define Partition Function
def Partition(value):
    end_value = pow(2,16)-1  # maximam value
    global length
    num_level = length
    return_value = math.ceil(value*num_level/end_value)
    #print(value,"  ", return_value);
    return return_value

All_Sample = []
for i in range(len(result)-1):
    a = list(result.keys())[i]
    a1 = result[a]["text"].split(":")[1][1:10]
    b = list(result.keys())[i+1]
    b1 = result[b]["text"].split(":")[1][1:10]
 
    if((a1==b1)):
        temp = result[a]["text"].split(":")[2]
        x1, x2 = temp.split(" ")[1], temp.split(" ")[2]
        y1, y2 = temp.split(" ")[3], temp.split(" ")[4]
        z1, z2 = temp.split(" ")[5], temp.split(" ")[6]
        #print(x1,x2,y1,y2,z1,z2)
        x = (int(x1)<<8) + int(x2)
        y = (int(y1)<<8) + int(y2)
        z = (int(z1)<<8) + int(z2)
        #print(x,y,z)
        sample = math.sqrt(x*x+y*y+z*z)
            # print(sample)
        sample = Partition(int(sample))
            #print(sample)
        All_Sample.append(sample)
        #Sample_list = ''.join([str(elem) for elem in All_Sample]) 
        
    if((a1==b1)==False or i==len(result)-2):
        print("\n")
        print(All_Sample)
        print(len(All_Sample))
        All_Sample = []



[9, 1, 8, 9, 2, 10, 9, 2, 10, 6, 10, 6, 8, 6, 8, 1, 3, 3, 6, 2, 2, 9, 10, 3, 8, 6, 3, 6, 9, 6, 5, 9, 7, 1, 10, 2, 10, 9, 9, 2, 10, 2, 10, 10, 5, 10, 10, 2, 2, 5, 2, 6, 3, 3, 3, 2, 8, 6, 6, 1, 8, 8, 6, 8, 9, 3, 10, 1, 7, 8, 1, 9, 9, 7, 6, 9, 6, 2, 1, 1, 8, 1, 7, 1, 7, 9, 2, 6, 2, 6, 9, 2, 9, 7, 9, 9, 9, 6, 9, 3, 8, 9, 3, 9, 6, 9, 3, 7, 3, 10, 9, 8, 8, 9, 9, 9, 1, 9, 3, 6, 6, 6, 3, 9, 8, 10, 9, 9, 6, 3, 8, 1, 9, 3, 9, 7, 7, 8, 1, 5, 2, 5, 9, 10, 2, 2, 6, 6, 1, 8, 3, 3, 9, 1, 3, 3, 9, 1, 1, 1, 6, 9, 6, 3, 9, 6, 9, 8, 3, 6, 3, 6, 6, 9, 6, 10, 3, 6, 1, 9, 7, 2, 10, 9, 7, 10, 2, 6, 2, 9, 5, 6, 3, 9, 1, 8, 7, 5, 8, 3, 6, 3, 9, 5, 9, 10, 2, 10, 6, 6, 9, 1, 9, 1, 8, 8, 9, 8, 6, 6, 8, 3, 1, 3, 3, 1, 6, 6, 6, 6, 3, 5, 10, 5, 1, 9, 8, 9, 1, 1, 1, 1, 5, 9, 1, 3, 6, 6, 8, 3, 3, 9, 3, 10, 3, 2, 6, 5, 5, 8, 6, 9, 1, 9, 3, 8, 3, 7, 3, 1, 3, 1, 9, 8, 10, 9, 7, 9, 10, 10, 9, 5, 6, 8, 9, 9, 9, 3, 9, 6, 9, 9, 10, 9, 1, 6, 9, 1, 5, 9, 10, 10, 6, 2, 7, 6, 3, 3, 1, 7, 1, 3, 9, 6, 3, 7, 8, 3, 8, 8, 3, 9, 10,